In [25]:
# データセット作成

!pip install scipy

window_size = 1024
window_num_per_clip = 40
clip_size = window_size * window_num_per_clip

from scipy.io.wavfile import read
import glob
import os
import numpy as np


if os.path.isfile("../data/arrays/clips.npy"):
    print("clips.npy exists.")
    clips = np.load("../data/arrays/clips.npy")
else:
    files = glob.glob("../data/wav44100/*")
    #files = ["../data/wav44100/3DEmbodimentFromLines.wav"]
    raw_data_list = [read(file)[1] for file in files]

    clips = np.zeros((0, clip_size), dtype=np.int16)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, clip_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save("../data/arrays/clips", clips)
    
if os.path.isfile("../data/arrays/fft.npy"):
    print("fft.npy exists.")
    fft = np.load("../data/arrays/fft.npy")
else:
    fft = np.zeros((0, window_num_per_clip, window_size))
    for clip in clips:
        fft_tmp = np.zeros((0, window_size))
        tmp = [clip[i:i + window_size] for i in range(0, len(clip) - window_size + 1, window_size)]
        for tmp2 in tmp:
            fft_tmp = np.vstack((fft_tmp, np.abs(np.fft.fft(tmp2))))
        fft = np.vstack((fft, fft_tmp.reshape(1, window_num_per_clip, window_size)))

    np.save("../data/arrays/fft", fft)

print(fft.shape)
print(type(fft[0][0][0]))
print(clips.shape)

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
clips.npy exists.
fft.npy exists.
(255, 40, 1024)
<class 'numpy.float64'>
(255, 40960)


array([ 6182.        , 10291.98714373, 48211.97434564, ...,
       45003.1621557 , 48211.97434564, 10291.98714373])

In [ ]:
# 学習データとテストデータに分ける

train_data_rate = 0.8

p = np.random.permutation(len(x))
fft = fft[p]
clips = clips[p]

x_train = fft[:int(fft.shape[0] * train_data_rate), :, :]
x_test = fft[int(fft.shape[0] * train_data_rate):-1, :, :]
y_train = fft[1:int(fft.shape[0] * train_data_rate) + 1, 0, :]
y_test = fft[int(fft.shape[0] * train_data_rate) + 1:, 0, :]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)